In [1]:
from collections import defaultdict
from gensim.models import FastText
from itertools import zip_longest
from tqdm import tqdm_notebook

In [2]:
model = FastText.load('../embedding/fasttext_300_12_1_50_sg0.bin')
index = 1
vocab_dict = defaultdict(int)
for vocab in model.wv.vocab:
    vocab_dict[vocab] = index
    index += 1
print(len(model.wv.vocab))

105792


In [3]:
def read_tsv(type_):
    data = []
    with open('../data/{}.tsv'.format(type_), encoding='utf-8') as file:
        for line in file:
            if type_.find('test') < 0:
                c1, c2, y = line.split('\t')
                data.append((c1.split(' '), c2.split(' '), y[:1]))
            else:
                c1, c2 = line.split('\t')
                data.append((c1.split(' '), c2.split(' ')))
    return data

In [4]:
def write_ctf(type_, data):
    word_size = 12
    pad_spec = '{0:<%d.%d}' % (word_size, word_size)
    sanitize = str.maketrans({"|": None, "\n": None})

    with open('../data/{}.ctf'.format(type_), 'w', encoding='utf-8') as file:
        for idx, d in tqdm_notebook(enumerate(data)):
            if type_.find('test') < 0:
                c1, c2, ys = d[0], d[1], d[2]
                for c1_token, c2_token, y in zip_longest(c1, c2, ys):
                    out = [str(idx)]
                    if c1_token is not None:
                        out.append('|# %s' % pad_spec.format(c1_token.translate(sanitize)))
                        out.append('|c1 {}:{}'.format(vocab_dict[c1_token], 1))
                    if c2_token is not None:
                        out.append('|# %s' % pad_spec.format(c2_token.translate(sanitize)))
                        out.append('|c2 {}:{}'.format(vocab_dict[c2_token], 1))
                    if y is not None:
                        out.append('|y %3d  ' % int(y))
                    file.write('\t'.join(out))
                    file.write('\n')
            else:
                c1, c2 = d[0], d[1]
                for c1_token, c2_token in zip_longest(c1, c2):
                    out = [str(idx)]
                    if c1_token is not None:
                        out.append('|# %s' % pad_spec.format(c1_token.translate(sanitize)))
                        out.append('|c1 {}:{}'.format(vocab_dict[c1_token], 1))
                    if c2_token is not None:
                        out.append('|# %s' % pad_spec.format(c2_token.translate(sanitize)))
                        out.append('|c2 {}:{}'.format(vocab_dict[c2_token], 1))
                    file.write('\t'.join(out))
                    file.write('\n')

In [5]:
train_data = read_tsv('train')
valid_data = read_tsv('dev')
test_data = read_tsv('test')

write_ctf('train', train_data)
write_ctf('dev', valid_data)
write_ctf('test', test_data)